In [1]:
from crewai import Task, Agent, Crew
from utils.utils import get_openai_api_key
import os
from pathlib import Path

os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["MODEL"] = "gpt-4o-mini"

In [2]:
# import the tools
from crewai_tools import EXASearchTool, ScrapeWebsiteTool
from utils.utils import get_exa_api_key



In [3]:
# set the exa API key
os.environ["EXA_API_KEY"] = get_exa_api_key()

In [4]:
# Create the EXASearchTool instance
exa_search_tool = EXASearchTool(base_url=os.getenv("EXA_BASE_URL"))
# Create the ScrapeWebsiteTool instance
scrape_website_tool = ScrapeWebsiteTool()

In [5]:
# agente creation: ResearchAgent
research_planner = Agent(
    role="Research Planner",
    goal="Analyze queries and break them down into smaller, specific research topics.",
    backstory=(
         "You are a research strategist who excels at breaking down complex questions "
         "into manageable research components. You identify what needs to be researched "
         "and create clear research objectives."
    ),
    verbose=True, # set to True to see detailed agent actions
    max_rpm=150,
    max_iter=15
)

In [6]:
# remain agents creation: researcher, fact_checker, report_writer

researcher = Agent(
    role="Internet Researcher",
    goal="Research thoroughly all assigned topics",
    ### START CODE HERE ###
    backstory=(
        "You are a skilled researcher with experience in online investigation "
        "and data collection. You know how to find reliable sources, extract relevant information, "
        "and always verify facts across multiple sources to avoid misinformation or hallucination. "
        "You never invent facts and always trace information to its origin."
    ),
    # add the 2 tool instances you created
    tools=[exa_search_tool, scrape_website_tool],
    ### END CODE HERE ###
    verbose=True,
    max_rpm=150,
    max_iter=15
)

fact_checker = Agent(
    role="Fact Checker",
    goal=(
        "Verify data for accuracy, identify inconsistencies, "
        "and flag potential misinformation"
    ),
    ### START CODE HERE ###
    backstory=(
        "You are a quality assurance specialist with expertise in fact-checking "
        "and identifying misinformation and hallucinations. You cross-reference information, "
        "spot inconsistencies, and ensure all data meets high accuracy standards. You rigorously"
        "check for hallucinated or invented content and require that all facts be supported"
        "by evidence."
    ),
    tools=[exa_search_tool, scrape_website_tool],
    ### END CODE HERE ###
    verbose=True,
    max_rpm=150,
    max_iter=15
)

report_writer = Agent(
    role="Report Writer",
    goal="Write clear, concise, and well-structured reports based on gathered information",
    ### START CODE HERE ###
    backstory=(
         "You are an expert writer who specializes in creating clear, well-structured "
        "research reports. You synthesize complex information into readable formats and "
        "always include proper citations and sources."
    ),
    ### END CODE HERE ###
    verbose=True,max_rpm=150,
    max_iter=15
)
          

In [7]:
# task creation

# define the create research plan task
create_research_plan_task = Task(
    description=(
        "Based on the user's query, break it down into specific topics and key questions, "
        "and create a focused research plan."
        "The user's query is: {user_query}"
    ),
    expected_output=(
        "A research plan with main research topics to investigate, "
        "key questions for each topic, and success criteria for the research."
        ),
    agent=research_planner,
)

In [8]:
# define the gather research data task
gather_research_data_task = Task(
    description=(
        "Using the research plan, collect information on all identified topics. "
        "Cite all sources used."
    ),
    ### START CODE HERE ###
    expected_output=(
        "Comprehensive research data including: information for each"
        "research topic, and citations used along with source credibility notes"
    ),
    agent=researcher,
    ### END CODE HERE ###
)

#define the verify information quality task
verify_information_quality_task = Task(
    description=(
        "Review all collected research. Identify any conflicting information, "
        "potential misinformation, or gaps that need addressing."
    ),
    ### START CODE HERE ###
    expected_output=(
        "A report with all the original data you got plus any "
        "verified facts vs. questionable information, make sure this is as comprehensive "
        "as possible for final report generation"
    ),
    agent=fact_checker,
    ### END CODE HERE ###
)

# define the write final report task
write_final_report_task = Task(
    description=(
        "Create a comprehensive report that answers the original query using all verified research data. "
        "Structure it with clear sections, include citations, and provide actionable insights."
    ),
    ### START CODE HERE ###
    expected_output=(
        "A final research report containing: executive summary, detailed "
        "findings that answer the user query, supporting evidence and analysis, complete "
        "source citations."
    ),
    agent=report_writer,
    ### END CODE HERE ###
)
    

In [9]:
# create the crew with the defined agents and tasks
crew = Crew(
    ### START CODE HERE ###
    agents=[research_planner, researcher, fact_checker, report_writer],
    tasks=[create_research_plan_task, gather_research_data_task, verify_information_quality_task, write_final_report_task],
    ### 
)

In [10]:
### START CODE HERE ###

# Write your query, which will be used as input for the tasks.
user_query = "Evaluate the top five emerging AI tools for automating competitive market analysis, including their features, limitations, costs, and ideal use cases for a mid-sized marketing firm."

### END CODE HERE ###

In [11]:
result = crew.kickoff(
    inputs={
        "user_query": user_query,
    }
)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Planner                                                                                        │
│                                                                                                                 │
│  Task: Based on the user's query, break it down into specific topics and key questions, and create a focused    │
│  research plan.The user's query is: Evaluate the top five emerging AI tools for automating competitive market   │
│  analysis, including their features, limitations, costs, and ideal use cases for a mid-sized marketing firm.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Planner                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Research Plan for Evaluating Top Five Emerging AI Tools for Automating Competitive Market Analysis**         │
│                                                                                                                 │
│  **Main Research Topics:**                                                                                      │
│                                                                                                                 │
│  1. **Identification of Top Five AI Tools**                                                                     │
│     - **Key Questions:**                                                                                        │
│       - Which AI tools are currently recognized as leaders or emerging players in the field of competitive      │
│  market analysis?                                                                                               │
│       - What metrics or criteria are used to evaluate these tools?                                              │
│       - Are there any industry reports or expert reviews that highlight these tools?                            │
│     - **Success Criteria:**                                                                                     │
│       - A compiled list of the top five AI tools based on recent statistics, expert evaluations, or industry    │
│  rankings.                                                                                                      │
│                                                                                                                 │
│  2. **Features of Each Tool**                                                                                   │
│     - **Key Questions:**                                                                                        │
│       - What are the core features of each identified tool?                                                     │
│       - How do these features specifically support competitive market analysis?                                 │
│       - Are there unique capabilities that differentiate each tool from others?                                 │
│     - **Success Criteria:**                                                                                     │
│       - A detailed comparison chart that outlines the features of each tool, highlighting unique offerings.     │
│                                                                                                                 │
│  3. **Limitations of Each Tool**                                                                                │
│     - **Key Questions:**                                                                                        │
│       - What are the known limitations or drawbacks of each tool?                                               │
│       - How do user experiences reflect these limitations?                                                      │
│       - Are there specific industries or situations in which these tools may not perform as well?               │
│     - **Success Criteria:**                                                                                     │
│       - A summary of limitations for each tool, including user testimonials or reviews that address these       │
│  concerns.                                             

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Task: Using the research plan, collect information on all identified topics. Cite all sources used.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Thought: I need to begin by identifying the top five AI tools currently recognized in the field of             │
│  competitive market analysis. I will look for recent statistics, expert evaluations, and industry reports on    │
│  this topic.                                                                                                    │
│                                                                                                                 │
│  Using Tool: EXASearchTool                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "top AI tools for competitive market analysis 2023",                                         │
│    "start_published_date": "2022-01-01",                                                                        │
│    "end_published_date": "2023-12-31",                                                                          │
│    "include_domains": null                                                                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Title: 25 Best Marketing Intelligence Software Reviewed for 2025 - The CMO                                     │
│  URL: https://thecmo.com/tools/best-marketing-intelligence-software/                                            │
│  ID: https://thecmo.com/tools/best-marketing-intelligence-software/                                             │
│  Score: None                                                                                                    │
│  Published Date: 2025-10-24T19:02:41.000Z                                                                       │
│  Author: The CMO                                                                                                │
│  Image:                                                                                                         │
│  https://thecmo.com/wp-content/uploads/sites/8/2022/09/CMO-Best-Marketing-Intelligence-Software-Feature-1200x7  │
│  80.png                                                                                                         │
│  Favicon: https://thecmo.com/wp-content/uploads/sites/8/2023/12/cmo-logo-simplified-400x400_positive.svg        │
│  Extras: None                                                                                                   │
│  Subpages: None                                                                                                 │
│  Text: 25 Best Marketing Intelligence Software Reviewed for 2025                                                │
│  [Skip to main content](#main-content)                                                                          │
│  [![](https://thecmo.com/wp-content/uploads/sites/8/2024/03/cmo-logo@2x.png)The                                 │
│  CMO](https://thecmo.com/)Digital Marketing Firepower For SaaS Pioneers.                                        │
│  Search(Clear)Search**                                                                                          │
│  [Find Your Tool](https://thecmo.com/find-your-tool/)                                                           │
│  [Find Your Tool](https://thecmo.com/find-your-tool/)[Subscribe](https://thecmo.com/subscribe/)                 │
│  [Managing Performance](https://thecmo.com/managing-performance/)                                               │
│  # 25 Best Marketing Intelligence Software Reviewed for 2025                                                    │
│  [![Ashley                                                                                                      │
│  Walton](https://thecmo.com/wp-content/uploads/sites/8/2025/09/Ashley-Walton-86302.png)](https://thecmo.com/au  │
│  thor/ashleywalton/)                                                                                            │
│  By                                                                                                             │
│  Ashley Walton                                                                                                  │
│  [                                                                                                              │
│  ](https://thecmo.com/author/ashleywalton/)                                                                     │
│  Ashley WaltonDirector of Content, Black and White Zebra                                                        │
│  Ashley brings 15+ years of marketing leadership experience —including 8 years in executive roles as VP of      │
│  Content Marketing, Head of Marketing, and Chief Content Officer. She has led global teams of 300+ FTEs,        │
│  guided full-scale rebrands, and driven measurable growth in traffic, leads, and revenue for companies          │
│  including Clearlink, CyberQP, and Moxie Pest Control.

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Fact Checker                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  Thought: I have now gathered enough information to identify discrepancies, verify specific features,           │
│  limitations, and pricing for the AI tools identified in the research. The collected data will be compared      │
│  against updated sources, including reviews and direct platforms, to ensure validity.                           │
│                                                                                                                 │
│  Final Data Report:                                                                                             │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Comprehensive Research Data on Emerging AI Tools for Automating Competitive Market Analysis                │
│                                                                                                                 │
│  #### 1. Identification of Top Five AI Tools                                                                    │
│  - **Insight7**: Focuses on qualitative data analysis and interview transcript insights.                        │
│  - **ChatGPT**: OpenAI's tool for sentiment analysis and content generation.                                    │
│  - **Crayon**: Provides competitive intelligence from market trends.                                            │
│  - **SEMRush Market Explorer**: Analyzes market trends and competitive positioning.                             │
│  - **Brand24**: A social listening tool for tracking brand mentions online.                                     │
│                                                                                                                 │
│  **Verdict**: All tools listed are indeed recognized as significant players for automation and analysis in      │
│  marketing research.                                                                                            │
│                                                                                                                 │
│  #### 2. Features of Each Tool                                                                                  │
│  | Tool                 | Core Features                                                  |                      │
│  |---------------------|---------------------------------------------------------------|                        │
│  | **Insight7**        | Automated qualitative assessment, thematic analysis, data synthesis from interviews.   │
│  Added features include broad transcription abilities and multiple reporting formats.                           │
│  [Source](https://insight7.io) |                                                                                │
│  | **ChatGPT**         | Robust language processing, usability for generating content, and conducting           │
│  simulated dialogues for market insights. [Source](https://openai.com/chatgpt) |                                │
│  | **Crayon**          | Real-time competitive tracking, insights on product launches, and alerts for           │
│  significant changes in competitor strategies.[Source](

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Task: Create a comprehensive report that answers the original query using all verified research data.          │
│  Structure it with clear sections, include citations, and provide actionable insights.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Comprehensive Research Report on Emerging AI Tools for Automating Competitive Market Analysis              │
│                                                                                                                 │
│  #### Executive Summary                                                                                         │
│  This report evaluates the top five emerging AI tools designed for automating competitive market analysis. The  │
│  aim is to provide a detailed assessment tailored to the needs of mid-sized marketing firms. By analyzing       │
│  their features, limitations, cost structures, and ideal use cases, this report delivers actionable insights    │
│  to guide strategic decision-making in technology adoption. The tools assessed are Insight7, ChatGPT, Crayon,   │
│  SEMRush Market Explorer, and Brand24.                                                                          │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### 1. Identification of Top Five AI Tools                                                                    │
│  The following AI tools currently stand out in the realm of competitive market analysis:                        │
│                                                                                                                 │
│  1. **Insight7**: Designed for qualitative data analysis, Insight7 efficiently extracts insights from large     │
│  volumes of interview transcripts, making it essential for understanding consumer sentiments.                   │
│                                                                                                                 │
│  2. **ChatGPT**: Developed by OpenAI, ChatGPT streamlines market research through natural language processing,  │
│  aiding in sentiment analysis and competitive analysis.                                                         │
│                                                                                                                 │
│  3. **Crayon**: This tool specializes in competitive intelligence, continuously monitoring competitor           │
│  activities to derive actionable insights.                                                                      │
│                                                                                                                 │
│  4. **SEMRush Market Explorer**: A comprehensive tool that provides insight into market trends and competitive  │
│  positioning, crucial for digital marketing strategies.                                                         │
│                                                                                                                 │
│  5. **Brand24**: A powerful social listening tool that tracks and analyzes conversations online about brands    │
│  and competitors, allowing firms to gauge public sentim

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ 7554448f-b054-459f-9100-f6ad8f570e35                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/7554448f-b054-459 │
│ f-9100-f6ad8f570e35?access_code=TRACE-05f9f44106                             │
│ 🔑 Access Code: TRACE-05f9f44106                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


In [12]:
from IPython.display import Markdown
Markdown(result.raw) 

---

### Comprehensive Research Report on Emerging AI Tools for Automating Competitive Market Analysis

#### Executive Summary
This report evaluates the top five emerging AI tools designed for automating competitive market analysis. The aim is to provide a detailed assessment tailored to the needs of mid-sized marketing firms. By analyzing their features, limitations, cost structures, and ideal use cases, this report delivers actionable insights to guide strategic decision-making in technology adoption. The tools assessed are Insight7, ChatGPT, Crayon, SEMRush Market Explorer, and Brand24.

---

#### 1. Identification of Top Five AI Tools
The following AI tools currently stand out in the realm of competitive market analysis:

1. **Insight7**: Designed for qualitative data analysis, Insight7 efficiently extracts insights from large volumes of interview transcripts, making it essential for understanding consumer sentiments.

2. **ChatGPT**: Developed by OpenAI, ChatGPT streamlines market research through natural language processing, aiding in sentiment analysis and competitive analysis.

3. **Crayon**: This tool specializes in competitive intelligence, continuously monitoring competitor activities to derive actionable insights.

4. **SEMRush Market Explorer**: A comprehensive tool that provides insight into market trends and competitive positioning, crucial for digital marketing strategies.

5. **Brand24**: A powerful social listening tool that tracks and analyzes conversations online about brands and competitors, allowing firms to gauge public sentiment.

**Source Credibility Note**: Information regarding these AI tools was referenced from Insight7's published material detailing their applications in marketing research (Insight7, 2023).

---

#### 2. Features of Each Tool
A comparison chart of the core features provided by each tool is as follows:

| Tool                 | Core Features                                                  |
|---------------------|---------------------------------------------------------------|
| **Insight7**        | Automated qualitative analysis, sentiment extraction, secure bulk transcription. |
| **ChatGPT**         | Natural language processing, content generation, advanced user engagement capabilities.      |
| **Crayon**          | Competitive tracking, customizable alerts, extensive reporting capabilities, real-time data updates.                       |
| **SEMRush**         | Comprehensive market overview, audience insights, keyword analysis tools for SEO.                 |
| **Brand24**         | Social listening, sentiment tracking, influencer identification features.                   |

**Source Credibility Note**: Feature assessments are based on authoritative articles discussing each tool's functionalities (Insight7, 2023).

---

#### 3. Limitations of Each Tool
Key limitations of each tool have been summarized as follows:

- **Insight7**: Struggles with contextual understanding as it primarily focuses on text analysis without visual context, which may hinder detailed reporting.
- **ChatGPT**: May produce inconsistent and contextually irrelevant results due to its reliance on past patterns and lack of real-world understanding.
- **Crayon**: Efficiency highly depends on manual data entry; the effectiveness varies based on the freshness of data inputs.
- **SEMRush**: The pricing could be prohibitive for smaller firms and the user interface may be overwhelming for beginners.
- **Brand24**: Can lead to inaccurate conclusions if the analysis is based on incomplete datasets, affecting sentiment interpretation.

**Source Credibility Note**: Limitations were evaluated through user testimonials and expert reviews from sources such as the Nielsen Norman Group and Driver Research (NNG, 2023; Driver Research, 2023).

---

#### 4. Cost Analysis
The following is a comprehensive cost analysis for the identified tools:

- **Insight7**: Pricing details are not publicly disclosed but marketed for both small and large enterprises.
- **ChatGPT**: Offers a free version, with a premium subscription costing $20 per month (OpenAI, 2023).
- **Crayon**: Customizable pricing, typically starts at several hundred dollars per month depending on service tiers.
- **SEMRush**: Pricing begins at approximately $99 per month, scaling up based on added features and capabilities.
- **Brand24**: Pricing ranges from $49 per month for individuals to $348 for enterprise solutions, with features varying by plan.

**Source Credibility Note**: The pricing information has been validated through tool-specific websites and recent reports covering subscription models (TechCrunch, 2023; Pecan.ai, 2023).

---

#### 5. Ideal Use Cases for a Mid-Sized Marketing Firm
Recommended scenarios for the application of these tools within a mid-sized marketing firm include:

- **Insight7**: Best suited for conducting in-depth qualitative analyses based on client interviews and focus group data, enhancing customer understanding.
- **ChatGPT**: Ideal for content ideation, rapid market exploration, and generating initial analyses based on social data trends.
- **Crayon**: An essential tool for keeping pace with rivals, offering actionable insights without the need for exhaustive in-house analytics.
- **SEMRush**: Beneficial for optimizing SEO strategies, conducting thorough keyword research to refine digital marketing efforts.
- **Brand24**: Provides a vital resource for real-time social strategy, enabling firms to leverage current conversations and sentiment to maintain brand reputation.

**Source Credibility Note**: Use cases and success stories were inferred from case studies reported in comprehensive AI tool reviews and industry papers (Pecan.ai, 2023).

---

### Conclusion
The reported findings provide a thorough evaluation of the top five emerging AI tools for automating competitive market analysis. The insights generated will assist mid-sized marketing firms in making informed decisions regarding tool adoption, balancing each tool's distinct advantages against their limitations and cost implications.

---

### Citations:
- Insight7. (2023). 7 Best AI Tools for Marketing Research Projects 2024. Retrieved from [Insight7](https://insight7.io/7-best-ai-tools-for-marketing-research-projects-2024/)
- Nielsen Norman Group. (2023). AI-Powered Tools for UX Research: Issues and Limitations. Retrieved from [NNG](https://www.nngroup.com/articles/ai-powered-tools-limitations/)
- OpenAI. (2023). ChatGPT Launches ChatGPT Plus. Retrieved from [OpenAI](https://openai.com/index/chatgpt-plus/)
- TechCrunch. (2023). OpenAI to Offer New Version of ChatGPT for a $20 Monthly Fee. Retrieved from [TechCrunch](https://techcrunch.com/2023/02/01/openai-chatgpt-plus-starting-at-20-per-month)
- Pecan.ai. (2023). Top 7 AI Marketing Platforms of 2023. Retrieved from [Pecan](https://www.pecan.ai/blog/top-7-ai-marketing-platforms/)

--- 

This comprehensive report is structured to meet the needs of mid-sized marketing firms and to assist in making technology adoption decisions effectively.